In [9]:
from httpx import get
import os

def get_response_for(keyword,results_per_page,page=1):
    url=f'https://unsplash.com/napi/search/photos?query={keyword}&per_page={results_per_page}&page={page}'
    response=get(url)
    if response.status_code==200:
        return response.json()
    
def get_img_urls(data):
    results=data['results']
    img_urls=[x['urls']['raw'] for x in results if x['premium'] is False]
    img_urls=[x.split('?')[0] for x in img_urls]
    return img_urls

def download_imgs(img_urls,max_download,dest_dir='images',tag=''):
    success_download=0
    for url in img_urls:
        if success_download<max_download:
            response=get(url)
            file_name=url.split('/')[-1]
            if not os.path.exists(dest_dir):
                os.makedirs(dest_dir)
            with open(f'{dest_dir}/{tag}{file_name}.jpeg','wb') as f:
                f.write(response.content)
                success_download+=1
        else:
            break
    return success_download

def scrape(keyword,num_of_results):
    start_page=1
    success_count=0
    while success_count<num_of_results :
        data=get_response_for(keyword,results_per_page=10,page=start_page)
        max_downloads=num_of_results-success_count
        if data:
            img_urls=get_img_urls(data)
            success_downloads=download_imgs(img_urls,max_downloads,tag=keyword)
            success_count+=success_downloads
            start_page+=1
        else:
            print('No Data Returned')
            break

data=get_response_for('dolphin',3)
print(get_img_urls(data))

['https://images.unsplash.com/photo-1547382442-d17c21625a44', 'https://images.unsplash.com/photo-1570481662006-a3a1374699e8']


In [10]:
scrape('running',3)